Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


In [3]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import heapq
import time  


PUZZLE_DIM = 3 
action = namedtuple('Action', ['pos1', 'pos2'])

def available_actions(state: np.ndarray) -> list:
    """Function to find available actions"""
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions


def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    """Function to perform an action"""
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state


def is_goal(state: np.ndarray) -> bool:
    """Function to check if the state is the goal"""
    goal = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    return np.array_equal(state, goal)


def manhattan_distance(state: np.ndarray) -> int:
    """Function to calculate Manhattan Distance heuristic"""
    total_distance = 0
    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            value = state[i, j]
            if value != 0:  # Ignore the empty tile
                goal_x, goal_y = divmod(value - 1, PUZZLE_DIM)
                total_distance += abs(i - goal_x) + abs(j - goal_y)
    return total_distance

# A* Algorithm
def a_star_search(initial_state: np.ndarray):
    """Solve the puzzle using the A* algorithm."""
    open_set = []
    heapq.heappush(open_set, (0, initial_state.tobytes(), [], 0))  # (priority, serialized state, path, cost)
    closed_set = set()

    while open_set:
        _, current_state_serialized, path, g = heapq.heappop(open_set)
        current_state = np.frombuffer(current_state_serialized, dtype=int).reshape((PUZZLE_DIM, PUZZLE_DIM))

        if is_goal(current_state):
            return path, g, len(closed_set)  

        closed_set.add(current_state_serialized)

        for action in available_actions(current_state):
            new_state = do_action(current_state, action)
            new_state_serialized = new_state.tobytes()
            if new_state_serialized not in closed_set:
                new_path = path + [action]
                h = manhattan_distance(new_state)
                heapq.heappush(open_set, (g + 1 + h, new_state_serialized, new_path, g + 1))

    return None, None, None  

# Initialize and randomize the puzzle state
RANDOMIZE_STEPS = 100
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for _ in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

# Display the initial state
print("Initial Randomized State:")
print(state)

# Solve the puzzle using A*
solution_path, quality, cost = a_star_search(state)

# Check if a solution was found
if solution_path is not None:
    print("\nSolution Steps:")
    for step_num, action in enumerate(solution_path, 1):
        print(f"\nStep {step_num}: Move tile from {action.pos2} to {action.pos1}")
        state = do_action(state, action)
        print(state)  
        time.sleep(0.5)  # Pause for visualization

    print("\nFinal Results:")
    print(f"Quality (solution length): {quality} moves")
    print(f"Cost (nodes evaluated): {cost}")
else:
    print("No solution found!")


Randomizing:   0%|          | 0/100 [00:00<?, ?it/s]

Initial Randomized State:
[[4 1 5]
 [2 0 7]
 [8 6 3]]

Solution Steps:

Step 1: Move tile from (1, 0) to (1, 1)
[[4 1 5]
 [0 2 7]
 [8 6 3]]

Step 2: Move tile from (0, 0) to (1, 0)
[[0 1 5]
 [4 2 7]
 [8 6 3]]

Step 3: Move tile from (0, 1) to (0, 0)
[[1 0 5]
 [4 2 7]
 [8 6 3]]

Step 4: Move tile from (1, 1) to (0, 1)
[[1 2 5]
 [4 0 7]
 [8 6 3]]

Step 5: Move tile from (1, 2) to (1, 1)
[[1 2 5]
 [4 7 0]
 [8 6 3]]

Step 6: Move tile from (0, 2) to (1, 2)
[[1 2 0]
 [4 7 5]
 [8 6 3]]

Step 7: Move tile from (0, 1) to (0, 2)
[[1 0 2]
 [4 7 5]
 [8 6 3]]

Step 8: Move tile from (0, 0) to (0, 1)
[[0 1 2]
 [4 7 5]
 [8 6 3]]

Step 9: Move tile from (1, 0) to (0, 0)
[[4 1 2]
 [0 7 5]
 [8 6 3]]

Step 10: Move tile from (1, 1) to (1, 0)
[[4 1 2]
 [7 0 5]
 [8 6 3]]

Step 11: Move tile from (1, 2) to (1, 1)
[[4 1 2]
 [7 5 0]
 [8 6 3]]

Step 12: Move tile from (2, 2) to (1, 2)
[[4 1 2]
 [7 5 3]
 [8 6 0]]

Step 13: Move tile from (2, 1) to (2, 2)
[[4 1 2]
 [7 5 3]
 [8 0 6]]

Step 14: Move tile from (2,